In [2]:
!pip install yfinance
import yfinance as yf #import api
import pandas as pd
from datetime import date
import datetime
import numpy as np
from datetime import  timedelta
import psycopg2
import plotly.express as px
import os

time = timedelta(days=1)
start_date  = date.today() - time #time difference as necessary will not show todays date due to time zone difference of american stocks
ticker = 'AMC'
# Get data

amc_data = yf.download(ticker,start=start_date,interval='1m')
amc_data = pd.DataFrame(amc_data)
amc_data['Company']= 'AMC'

ticker = 'FOX'
fox_data = yf.download(ticker, start_date,interval='1m')
fox_data = pd.DataFrame(fox_data)
fox_data['Company']= 'FOX'

ticker = 'T'
t_data = yf.download(ticker, start_date,interval='1m')
t_data = pd.DataFrame(t_data)
t_data['Company']= 'AT&T'

whole_df = pd.concat([amc_data,fox_data,t_data])

"""I wanted to add them together just using ticker = 'FOX AMC' however it was creating double headers which I was unsure of how to fix so instead I concatenated the two data frames into one big one in a simpler fashion"""

#Lets do some cleaning
whole_df.dtypes
#types look good
whole_df['date'] = whole_df.index #i want to create a new column replicating the index date isnt viable for index
#No index? lets change that
whole_df.reset_index(drop=True, inplace=True)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html - documentation used for resetting the index
#column names should be lower for efficiency and lets scrap the space from adj close!

#clean column names to fit quering for later use

new_columns = []

for i in whole_df.columns:
    new_name=i
    new_name= new_name.lower() #uncaptialise
    new_name=new_name.strip() #strip blanks
    new_name = new_name.replace(' ','_') #replace spacing between letters with a _ rather than having two or more words with spaces
    new_columns.append(new_name) #add the new column names into the column list

whole_df.columns = new_columns #update! lets assign our new columns back to our dataframe
#i want the full name as i dont have a dictionary explaining each of the columns
whole_df.rename(columns={'adj_close':'adjusted_close'}, inplace=True)
#i want a nice query lets split the date and time into two different components so i can query easier later on
whole_df['new_time']=pd.to_datetime(whole_df['date']).dt.time
whole_df['date'] = pd.to_datetime(whole_df['date']).dt.date

#I want to get only the day without any other day added as the api does not allow only a days worth will show me all results from the start date till now
mask=whole_df['date'] == pd.to_datetime(start_date).date()
whole_df = whole_df[mask] #assign it back
#lets view and see if it works

## Load into database
user = os.environ.get('userr')
password = os.environ.get('password')


#lets try our connection
try:
  connection = psycopg2.connect(

        user,
        password,
        host = "data-sandbox.c1tykfvfhpit.eu-west-2.rds.amazonaws.com",
        database = "pagila",
        port = "5432")
  cursor = connection.cursor()
  create_table = ''' CREATE TABLE student.pm_stocks (

                  open float,
                  high float,
                  low float,
                  close float,
                  adjusted_close float,
                  volume float,
                  company VARCHAR(20),
                  date DATE,
                  new_time TIME)
      '''
  # cursor.execute(create_table)
  #i already have the table i dont want to create another
  cursor.execute( 'SELECT date FROM student.pm_stocks WHERE date =%s',[start_date])
  df = pd.DataFrame(cursor.fetchall(), columns = ['date'])
  #df.loc[df['date'] == start_date,'check'] = 'True'
  #df.loc[df['date'] != start_date,'check'] = 'False'
  #my_list =df['check'].tolist()

  #my_list
  #for i in my_list:
  if df.empty:
 # dataframe = pd.read_csv('/content/2023-08-17_stocks.csv') importing weekday stocks to populate database (prior using storage)
      tups = list(whole_df.itertuples(index=False))
      query='INSERT INTO student.pm_stocks(open,high,low,close,adjusted_close,volume,company,date,new_time) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)'
      cursor.executemany(query,tups)
      connection.commit()
  else:
      pass
except Exception as error:
  print(error)
finally:
  connection.close()
#i want to close my connection to stop it from continously running best practise

try:
  connection = psycopg2.connect(
    user,
    password,
    host = "data-sandbox.c1tykfvfhpit.eu-west-2.rds.amazonaws.com",
    database = "pagila",
    port = "5432")
  cursor = connection.cursor()
  cursor.execute('select new_time,date,close,company from student.pm_stocks WHERE date = %s ',[start_date])
  #for record in cursor.fetchall():
  df = pd.DataFrame(cursor.fetchall(), columns = ['time', 'date', 'close','company'])
  #df.to_csv('query.csv',index=False)
  fig = px.line(df, x="time", y="close", title='Close stock prices comparison of FOX, HBO and AMC',color='company', symbol="company")
  fig.show()

  cursor.execute("select new_time,date,close,company from student.pm_stocks WHERE date = %s and company='FOX' ",[start_date])
  df = pd.DataFrame(cursor.fetchall(), columns = ['time', 'date', 'close','company'])
  fig = px.line(df, x="time", y="close", title='Close stock prices of FOX',color='company', symbol="company")
  fig.show()

  cursor.execute("select new_time,date,close,company from student.pm_stocks WHERE date = %s and company='AMC' ",[start_date])
  df = pd.DataFrame(cursor.fetchall(), columns = ['time', 'date', 'close','company'])
  fig = px.line(df, x="time", y="close", title='Close stock prices AMC',color='company', symbol="company")
  fig.show()


  cursor.execute("select new_time,date,close,company from student.pm_stocks WHERE date = %s and company='AT&T' ",[start_date])
  df = pd.DataFrame(cursor.fetchall(), columns = ['time', 'date', 'close','company'])
  fig = px.line(df, x="time", y="close", title='Close stock prices HBO',color='company', symbol="company")
  fig.show()
except Exception as error:
  print(error)
finally:
  connection.close()

whole_df.to_csv(str(start_date)+'_stocks.csv',index=False)
#i want each file to be named after the day it was taken from

[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AMC']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2023-11-26 -> 2023-11-27 04:24:06-05:00)')



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FOX']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2023-11-26 -> 2023-11-27 04:24:07-05:00)')



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['T']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2023-11-26 -> 2023-11-27 04:24:25-05:00)')
